In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_vertexai import VertexAIEmbeddings
from pathlib import Path
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredExcelLoader, Docx2txtLoader

import pickle
from langchain_chroma import Chroma

In [2]:
current_dir =Path.cwd() 
persist_directory = current_dir /'db'/ 'main_db'
persist_directory=str(persist_directory)

In [3]:
from langchain_google_vertexai import VertexAIEmbeddings
embeddings = VertexAIEmbeddings(model="text-embedding-004")

In [4]:
from langchain_chroma import Chroma

vector_store = Chroma(
    
    embedding_function=embeddings,
    persist_directory=persist_directory,  # Where to save data locally, remove if not necessary
)

In [5]:

def upload_Embeddings():
    """Function to process files from the 'uploads' directory and store embeddings."""
    
    # Define paths

    uploads_path = current_dir / 'uploads'
    # uploads_path =r"C://Users//DELL//Desktop//CloneFlow//backend//uploads"
    
    print("Current Directory:", current_dir)


    # Get all files in the uploads directory
    files = list(uploads_path.glob("*"))  # ✅ More robust than os.listdir()

    for f in files:
        
            ext = f.suffix.lower().lstrip('.')  # ✅ Get file extension safely

            # Load document based on file type
            if ext == 'txt':
                print(f'{f.name} ---> is a txt')
                loader = TextLoader(str(f))
            elif ext == 'pdf':
                print(f'{f.name} ---> is a pdf')
                loader = PyPDFLoader(str(f))
            elif ext == 'docx':
                print(f'{f.name} ---> is a docx')
                loader = Docx2txtLoader(str(f))
            elif ext == 'xlsx':
                print(f'{f.name} ---> is an xlsx')
                loader = UnstructuredExcelLoader(str(f))
            else:
                print(f'{f.name} ---> skipped')
                continue  

            document = loader.load()
         

            # Split the document into chunks
            text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
            docs = text_splitter.split_documents(document)

            if not docs:
                print(f"Split into {len(docs)} chunks, so skipped")
                continue

            print(f"Split into {len(docs)} chunks")

            # Generate embeddings and store in Chroma
            embeddings = VertexAIEmbeddings(model="text-embedding-004")
            vector_store= Chroma.from_documents(docs, embeddings, persist_directory=str(persist_directory))
            
   
                
            print(f'Done with embeddings for {f}')
       
            



In [6]:

upload_Embeddings()

Current Directory: c:\Users\DELL\Desktop\CloneFlow\production
dataBasePassword.txt ---> is a txt
Split into 1 chunks
Done with embeddings for c:\Users\DELL\Desktop\CloneFlow\production\uploads\dataBasePassword.txt
requirements.txt ---> is a txt
Split into 1 chunks


Created a chunk of size 1067, which is longer than the specified 1000


Done with embeddings for c:\Users\DELL\Desktop\CloneFlow\production\uploads\requirements.txt
youtubeinfo.txt ---> is a txt
Split into 26 chunks
Done with embeddings for c:\Users\DELL\Desktop\CloneFlow\production\uploads\youtubeinfo.txt


In [7]:
resoure = vector_store.similarity_search("youtube", k=3)
print(resoure)

[Document(id='f9d20cb2-9ba5-4c15-82c4-efcba0677b48', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\uploads\\youtubeinfo.txt'}, page_content='Several YouTube channels have gained massive followings, with T-Series being the most-subscribed channel, followed by MrBeast, Cocomelon, and PewDiePie. The platform has also played a crucial role in launching the careers of content creators, musicians, and influencers.\n\nYouTube has faced multiple challenges over the years, including concerns about copyright infringement, misinformation, privacy issues, and content moderation. The company has implemented policies to combat harmful content, including demonetization of inappropriate videos, stricter copyright enforcement through Content ID, and improved moderation using AI and human reviewers.\n\nIn addition to its main platform, YouTube offers YouTube Music, a streaming service for music and music videos; YouTube TV, a live TV streaming service offering cable channels; and

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI   
from langchain_core.output_parsers import StrOutputParser     
from dotenv import load_dotenv
load_dotenv()
from uuid import uuid4
import os
llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash'
)

In [ ]:
def json_to_txt(file_path:str):
    '''Makes json in txt format using ai if already in text  format returns the same text'''
    json_content=[]
    
    with open(file_path, 'r') as file:
        json_content = file.read()
        
    response = llm.invoke(f"You are ai whose job is express json in form of sententes:{json_content}",)
    with open(file_path, 'w') as file:
        file.write(response.content)
        
        
    print("response",response.content) 
      
    

In [10]:
# json_to_txt(r"C://Users//DELL\Desktop//CloneFlow//production//user_settings//user_info.txt")

In [11]:
def establish_relation():
          # Define paths
    user_settings = current_dir / 'user_settings' 
    
    files = list((user_settings.glob("*"))) 
    
    print("File names in user_settings:", files)
    
    uuids = [str(uuid4()) for _ in range(len(files))]  # Generate UUIDs for each file
    relationships = {}
    for i in range(len(files)):
        relationships[files[i].name] =uuids[i]
        
    print("UUIDs:", uuids)
    print("Relationships:", relationships)
    

In [12]:
# establish_relation()

In [13]:
# current_dir = Path.cwd()   # ✅ Ensures correct current directory
# persist_directory = current_dir / 'db' / 'chroma_db'
# user_settings = current_dir / 'user_settings' 
# files = list((user_settings.glob("*"))) 
# docslist = []
# for f in files:
    
#         json_to_txt(str(f))  # Convert JSON to text format
#         print(f)
#         loader = TextLoader(str(f)) 
#         document = loader.load()
#         text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
#         docs = text_splitter.split_documents(document)
#         docslist.append(docs)
#         print("docs",docs)
        

In [14]:
def storing_vector_user_settings():
    """Function to  store vector embeddings for user settings."""
        # Define paths

    user_settings = current_dir / 'user_settings' 
    files = list((user_settings.glob("*"))) 
    
    # relationships = {}
    
    # # making ids for each file
    # ids= [x for x in range(len(files))]
    # ids= [str(x) for x in ids]  # Convert to string
    
    # for i in range(len(files)):
    #     relationships[files[i].name] =ids[i]
        
        
    for f in files:
        # json_to_txt(str(f))  # Convert JSON to text format
        print(f)
        loader = TextLoader(str(f)) 
        document = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
        docs = text_splitter.split_documents(document)

    
        if not docs:
         print(f"Split into {len(docs)} chunks, so skipped")
         continue

        print(f"Split into {len(docs)} chunks")
        
    
        vector_store= Chroma.from_documents(docs, embeddings, persist_directory=str(persist_directory))
        
    
 
    
 

In [15]:

user_settings = current_dir / 'user_settings' 
files = list((user_settings.glob("*"))) 
    
relationships = {}
    
    # making ids for each file
ids= [x for x in range(len(files))]
ids= [str(x) for x in ids]  # Convert to string
    
for i in range(len(files)):
     relationships[files[i].name] =ids[i]
relationships    
        

{'contact.txt': '0', 'instruction.txt': '1', 'user_info.txt': '2'}

In [16]:
# storing_vector_user_settings()

In [ ]:
def docs_list():
    from itertools import chain
    """Function to load the vector store from the database."""
        # Define paths

    user_settings = current_dir / 'user_settings' 
    files = list((user_settings.glob("*"))) 
    print([os.path.basename(f) for f in files])
    docslist = []
    
    for f in files:
        # json_to_txt(str(f))  # Convert JSON to text format
        # print(f)
        loader = TextLoader(str(f)) 
        document = loader.load()
        
        if not document:
            print(f"Split into {len(document)} chunks, so skipped")
            continue
           
        if os.path.basename(f) == 'contact.txt':
            text_splitter = CharacterTextSplitter(chunk_size=5, chunk_overlap=10)
            docs = text_splitter.split_documents(document)
            print("docs",docs)
            
            
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
        docs = text_splitter.split_documents(document)
        docslist.append(docs)
        # print("docs",docs)
        
        
    print("docslist",docslist)    
    return list(chain.from_iterable(docslist))

In [34]:
docs_list()

['contact.txt', 'instruction.txt', 'user_info.txt']
docslist [[Document(metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\contact.txt'}, page_content='This is a contact for Mammi. You can reach them at +919370300989.\nThis is a contact for Papa. You can reach them at 982-380-4211.\nThis is a contact for Pranav Dipak Khadse. You can reach them at +919960657036.\nThis is a contact for Gorakh Kadam. You can reach them at +918605799757.\nThis is a contact for Parth Mahajan. You can reach them at +919665735058.\nThis is a contact for Simaya Travels. You can reach them at 744-778-1133.\nThis is a contact for Jalindar Kadam. You can reach them at +917738252323.\nThis is a contact for Rohan Shinde (Database Head- Abhivriddhi). You can reach them at 800-767-5714.\nThis is a contact for Shreya Shelke (Design Head- Abhivriddhi). You can reach them at 928-460-0997.\nThis is a contact for Parth Kunghatkar (Design Head- Abhivriddhi). You can reach them at 956-190-6

[Document(metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\contact.txt'}, page_content='This is a contact for Mammi. You can reach them at +919370300989.\nThis is a contact for Papa. You can reach them at 982-380-4211.\nThis is a contact for Pranav Dipak Khadse. You can reach them at +919960657036.\nThis is a contact for Gorakh Kadam. You can reach them at +918605799757.\nThis is a contact for Parth Mahajan. You can reach them at +919665735058.\nThis is a contact for Simaya Travels. You can reach them at 744-778-1133.\nThis is a contact for Jalindar Kadam. You can reach them at +917738252323.\nThis is a contact for Rohan Shinde (Database Head- Abhivriddhi). You can reach them at 800-767-5714.\nThis is a contact for Shreya Shelke (Design Head- Abhivriddhi). You can reach them at 928-460-0997.\nThis is a contact for Parth Kunghatkar (Design Head- Abhivriddhi). You can reach them at 956-190-6218.\nThis is a contact for Shreyas Chulliyan (Design Head- Ab

In [19]:
vector_store= Chroma.from_documents(docs_list(), embeddings, persist_directory=str(persist_directory),ids=ids)

c:\Users\DELL\Desktop\CloneFlow\production\user_settings\contact.txt
docs [Document(metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\contact.txt'}, page_content='This is a contact for Mammi. You can reach them at +919370300989.\nThis is a contact for Papa. You can reach them at 982-380-4211.\nThis is a contact for Pranav Dipak Khadse. You can reach them at +919960657036.\nThis is a contact for Gorakh Kadam. You can reach them at +918605799757.\nThis is a contact for Parth Mahajan. You can reach them at +919665735058.\nThis is a contact for Simaya Travels. You can reach them at 744-778-1133.\nThis is a contact for Jalindar Kadam. You can reach them at +917738252323.\nThis is a contact for Rohan Shinde (Database Head- Abhivriddhi). You can reach them at 800-767-5714.\nThis is a contact for Shreya Shelke (Design Head- Abhivriddhi). You can reach them at 928-460-0997.\nThis is a contact for Parth Kunghatkar (Design Head- Abhivriddhi). You can reach them 

In [20]:
retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1}, 
)

In [21]:
relevant_docs = retriever.invoke("youtube")
print(relevant_docs)

[Document(id='f9d20cb2-9ba5-4c15-82c4-efcba0677b48', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\uploads\\youtubeinfo.txt'}, page_content='Several YouTube channels have gained massive followings, with T-Series being the most-subscribed channel, followed by MrBeast, Cocomelon, and PewDiePie. The platform has also played a crucial role in launching the careers of content creators, musicians, and influencers.\n\nYouTube has faced multiple challenges over the years, including concerns about copyright infringement, misinformation, privacy issues, and content moderation. The company has implemented policies to combat harmful content, including demonetization of inappropriate videos, stricter copyright enforcement through Content ID, and improved moderation using AI and human reviewers.\n\nIn addition to its main platform, YouTube offers YouTube Music, a streaming service for music and music videos; YouTube TV, a live TV streaming service offering cable channels; and

In [22]:
relevant_docs = retriever.invoke(
    '''partha'''
)
print(relevant_docs)

[Document(id='39c3f35d-4d3f-472a-94d9-8160f8792523', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\uploads\\dataBasePassword.txt'}, page_content='AI Email with Partha      J9wIP7bZWTCSg0L4'), Document(id='1', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\user_info.txt'}, page_content='Partha Kadam, a seventeen-year-old living at 123 Main Street in Springfield, USA, can be reached at partha.kadam@example.com or 123-456-7890.'), Document(id='0', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\contact.txt'}, page_content='This is a contact for Mammi. You can reach them at +919370300989.\nThis is a contact for Papa. You can reach them at 982-380-4211.\nThis is a contact for Pranav Dipak Khadse. You can reach them at +919960657036.\nThis is a contact for Gorakh Kadam. You can reach them at +918605799757.\nThis is a contact for Parth Mahajan. You can reach them at +919665735058.\nThis is a con

In [23]:
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
AI Email with Partha      J9wIP7bZWTCSg0L4

Source: c:\Users\DELL\Desktop\CloneFlow\production\uploads\dataBasePassword.txt

Document 2:
Partha Kadam, a seventeen-year-old living at 123 Main Street in Springfield, USA, can be reached at partha.kadam@example.com or 123-456-7890.

Source: c:\Users\DELL\Desktop\CloneFlow\production\user_settings\user_info.txt

Document 3:
This is a contact for Mammi. You can reach them at +919370300989.
This is a contact for Papa. You can reach them at 982-380-4211.
This is a contact for Pranav Dipak Khadse. You can reach them at +919960657036.
This is a contact for Gorakh Kadam. You can reach them at +918605799757.
This is a contact for Parth Mahajan. You can reach them at +919665735058.
This is a contact for Simaya Travels. You can reach them at 744-778-1133.
This is a contact for Jalindar Kadam. You can reach them at +917738252323.
This is a contact for Rohan Shinde (Database Head- Abhivriddhi). You can reach the

In [ ]:
# vector_store.delete(ids="36ce41ef-f62a-480f-8c5d-f6faa1bb48fe")
